# Riskfolio-Lib Tutorial: 
<br>__[Financionerioncios](https://financioneroncios.wordpress.com)__
<br>__[Orenji](https://www.orenj-i.net)__
<br>__[Riskfolio-Lib](https://riskfolio-lib.readthedocs.io/en/latest/)__
<br>__[Dany Cajas](https://www.linkedin.com/in/dany-cajas/)__
<a href='https://ko-fi.com/B0B833SXD' target='_blank'><img height='36' style='border:0px;height:36px;' src='https://cdn.ko-fi.com/cdn/kofi1.png?v=2' border='0' alt='Buy Me a Coffee at ko-fi.com' /></a> 

## Tutorial 1: Classic Mean Risk Optimization

## 1. Downloading the data:

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import warnings

warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.4%}'.format

# Date range
start = '2022-01-03'
end = '2025-03-21'

# Tickers of assets
mag7 = ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA']
AI_beneficiary = ['SNOW', 'DDOG', 'CRM']
sectors = ['XLC', 'XLB', 'XLV', 'XLRE', 'XLK', 'XLY', 'XLP', 'XLF', 'XLE', 'XLI', 'XLU', 'MID', 'EEM', 'GLD'] # S&P 500 sectors]
assets = sectors + AI_beneficiary #+ mag7
assets.sort()

In [ ]:
# Downloading data
data = yf.download(assets, start = start, end = end)
data = data.loc[:,('Close', slice(None))]
data.columns = assets

In [ ]:
# Calculating returns

Y = data[assets].pct_change().dropna()

display(Y.head())

## 2. Estimating Mean Variance Portfolios

### 2.1 Calculating the portfolio that maximizes Sharpe ratio.

In [ ]:
import riskfolio as rp

# Building the portfolio object
port = rp.Portfolio(returns=Y)

# Calculating optimal portfolio

# Select method and estimate input parameters:

method_mu='hist' # Method to estimate expected returns based on historical data.
method_cov='hist' # Method to estimate covariance matrix based on historical data.

port.assets_stats(method_mu=method_mu, method_cov=method_cov)

# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'MV' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

### 2.2 Plotting portfolio composition

In [ ]:
# Plotting the composition of the portfolio

ax = rp.plot_pie(w=w, title='Sharpe Mean Variance', others=0.05, nrow=25, cmap = "tab20",
                 height=6, width=10, ax=None)

### 2.3 Calculate efficient frontier

In [ ]:
points = 50 # Number of points of the frontier

frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

display(frontier.T.head())

In [ ]:
# Plotting the efficient frontier

label = 'Max Risk Adjusted Return Portfolio' # Title of point
mu = port.mu # Expected returns
cov = port.cov # Covariance matrix
returns = port.returns # Returns of the assets

ax = rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm,
                      rf=rf, alpha=0.05, cmap='viridis', w=w, label=label,
                      marker='*', s=16, c='r', height=6, width=10, ax=None)

In [ ]:
# Plotting efficient frontier composition

ax = rp.plot_frontier_area(w_frontier=frontier, cmap="tab20", height=6, width=10, ax=None)

## 3. Estimating Mean Risk Portfolios

In this part I will calculate optimal portfolios for several risk measures. First I'm going to calculate the portfolio that maximizes risk adjusted return when CVaR is the risk measure, then I'm going to calculate the portfolios that maximize the risk adjusted return for all available risk measures.

### 3.1 Calculating the portfolio that maximizes Return/CVaR ratio.

In [ ]:
rm = 'CVaR' # Risk measure

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

### 3.2 Plotting portfolio composition

In [ ]:
ax = rp.plot_pie(w=w, title='Sharpe Mean CVaR', others=0.05, nrow=25, cmap = "tab20",
                 height=6, width=10, ax=None)

### 3.3 Calculate efficient frontier

In [ ]:
points = 50 # Number of points of the frontier

frontier = port.efficient_frontier(model=model, rm=rm, points=points, rf=rf, hist=hist)

display(frontier.T.head())

In [ ]:
label = 'Max Risk Adjusted Return Portfolio' # Title of point

ax = rp.plot_frontier(w_frontier=frontier, mu=mu, cov=cov, returns=returns, rm=rm,
                      rf=rf, alpha=0.05, cmap='viridis', w=w, label=label,
                      marker='*', s=16, c='r', height=6, width=10, ax=None)

In [ ]:
# Plotting efficient frontier composition

ax = rp.plot_frontier_area(w_frontier=frontier, cmap="tab20", height=6, width=10, ax=None)

### 3.4 Calculate Optimal Portfolios for Several Risk Measures

In [ ]:
# Risk Measures available:
#
# 'MV': Standard Deviation.
# 'MAD': Mean Absolute Deviation.
# 'MSV': Semi Standard Deviation.
# 'FLPM': First Lower Partial Moment (Omega Ratio).
# 'SLPM': Second Lower Partial Moment (Sortino Ratio).
# 'CVaR': Conditional Value at Risk.
# 'EVaR': Entropic Value at Risk.
# 'WR': Worst Realization (Minimax)
# 'MDD': Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio).
# 'ADD': Average Drawdown of uncompounded cumulative returns.
# 'CDaR': Conditional Drawdown at Risk of uncompounded cumulative returns.
# 'EDaR': Entropic Drawdown at Risk of uncompounded cumulative returns.
# 'UCI': Ulcer Index of uncompounded cumulative returns.

rms = ['MV', 'MAD', 'MSV', 'FLPM', 'SLPM', 'CVaR',
       'EVaR', 'WR', 'MDD', 'ADD', 'CDaR', 'UCI', 'EDaR']

w_s = pd.DataFrame([])

for i in rms:
    w = port.optimization(model=model, rm=i, obj=obj, rf=rf, l=l, hist=hist)
    w_s = pd.concat([w_s, w], axis=1)
    
w_s.columns = rms

In [ ]:
w_s.style.format("{:.2%}").background_gradient(cmap='YlGn')

In [ ]:
import matplotlib.pyplot as plt

# Plotting a comparison of assets weights for each portfolio

fig = plt.gcf()
fig.set_figwidth(14)
fig.set_figheight(6)
ax = fig.subplots(nrows=1, ncols=1)

w_s.plot.bar(ax=ax)

## 4. Constraints on Assets and Assets Classes

### 4.1 Creating the constraints

In this part I use dictionaries to create the constraints but is prefered to create the tables in excel and upload them with pandas.read_excel.

In [ ]:


# Define assets and their corresponding industries
asset_classes = {
    'Assets': [#'AAPL', 'AMZN', 'GOOG', 'META', 'MSFT', 'NVDA', 'TSLA', # Magnificent 7
              'XLC', 'XLB', 'XLV', 'XLRE', 'XLK', 'XLY', 'XLP', 'XLF', 'XLE', 'XLI', 'XLU', 'MID', 'EEM', 'GLD', 
              # AI Beneficiaries
              'CRM', 'DDOG', 'SNOW'],
    "AI Beneficiary": ['No','No','No','No','No','No','No','No','No','No','No','No','No','No','Yes','Yes','Yes'],
    'Industry': [#'Technology', 'Consumer Discretionary', 'Communication Services', 'Communication Services', 'Technology', 'Technology', 'Consumer Discretionary', # Magnificent 7
                 'Communication Services', 'Materials', 'Healthcare', 'Real Estate',
                 'Technology', 'Consumer Discretionary', 'Consumer Staples', 'Financials', 'Energy', 'Industrials', 
                 'Utilities', 'Mid-Cap','Emerging Markets', 'Gold', 'Technology', 'Technology', 'Technology'],
}

# Create the 'asset_classes' DataFrame
asset_classes = pd.DataFrame(asset_classes)
asset_classes = asset_classes.sort_values(by=['Assets'])

# Define the constraints based on Goldman Sachs' recommendations
# Define the constraints based on Goldman Sachs' recommendations
gs_constraints = [
    # Sector Overweighting (using "Classes" and relative to benchmark)
    {"Type": "Classes", "Set": "Industry", "Position": "Materials", "Weight": 0.019, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""}, 
    {"Type": "Classes", "Set": "Industry", "Position": "Energy", "Weight": 0.05, "Sign": "<=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Classes", "Set": "Industry", "Position": "Healthcare", "Weight": 0.101, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""}, 
    {"Type": "Classes", "Set": "Industry", "Position": "Real Estate", "Weight": 0.023, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""}, 
    {"Type": "Classes", "Set": "Industry", "Position": "Technology", "Weight": 0.29, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Classes", "Set": "Industry", "Position": "Financials", "Weight": 0.136, "Sign": "<=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Classes", "Set": "Industry", "Position": "Communication Services", "Weight": 0.082, "Sign": "<=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Classes", "Set": "Industry", "Position": "Consumer Discretionary", "Weight": 0.113, "Sign": "<=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Classes", "Set": "Industry", "Position": "Mid-Cap", "Weight": 0.1, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Classes", "Set": "Industry", "Position": "Emerging Markets", "Weight": 0.025, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Classes", "Set": "Industry", "Position": "Gold", "Weight": 0.05, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Classes", "Set": "AI Beneficiary", "Position": "Yes", "Weight": 0.015, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    {"Type": "Each asset in a class", "Set": "AI Beneficiary", "Position": "Yes", "Weight": 0.02, "Sign": "<=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    #{"Type": "Asset", "Set": "", "Position": "SNOW", "Weight": 0.1, "Sign": ">=", "Relative": "", "Type Relative": "", "Relative Set": "", "Factor":""},
    #{"Type": "All Assets", "Set": "", "Position": "AAPL", "Sign": "<=", "Weight": 0.0758, "Relative": "Technology", "Type Relative": "Classes", "Relative Set": "Industry", "Factor": 1.0},
    #{"Type": "All Assets", "Set": "", "Position": "AMZN", "Sign": "<=", "Weight": 0.0411, "Relative": "Consumer Discretionary", "Type Relative": "Classes", "Relative Set": "Industry", "Factor": 1.0},
    #{"Type": "All Assets", "Set": "", "Position": "GOOG", "Sign": "<=", "Weight": 0.0402, "Relative": "Communication Services", "Type Relative": "Classes", "Relative Set": "Industry", "Factor": 1.0},
    #{"Type": "All Assets", "Set": "", "Position": "META", "Sign": "<=", "Weight": 0.0255, "Relative": "Communication Services", "Type Relative": "Classes", "Relative Set": "Industry", "Factor": 1.0},
    #{"Type": "All Assets", "Set": "", "Position": "MSFT", "Sign": "<=", "Weight": 0.0627, "Relative": "Technology", "Type Relative": "Classes", "Relative Set": "Industry", "Factor": 1.0},
    #{"Type": "All Assets", "Set": "", "Position": "NVDA", "Sign": "<=", "Weight": 0.0659, "Relative": "Technology", "Type Relative": "Classes", "Relative Set": "Industry", "Factor": 1.0},
    #{"Type": "All Assets", "Set": "", "Position": "TSLA", "Sign": "<=", "Weight": 0.0226, "Relative": "Consumer Discretionary", "Type Relative": "Classes", "Relative Set": "Industry", "Factor": 1.0},
]

# Create the 'constraints' DataFrame
constraints = pd.DataFrame(gs_constraints)

# Add the remaining columns with default values
constraints['Disabled'] = False
constraints['Position'] = constraints.get('Position', None)  
constraints['Relative Set'] = constraints.get('Relative Set', None)
constraints['Factor'] = constraints.get('Factor', None)

# Reorder columns to match the example
constraints = constraints[['Disabled', 'Type', 'Set', 'Position', 'Sign', 'Weight', 'Type Relative', 'Relative Set', 'Relative', 'Factor']]

# Display the 'constraints' DataFrame
#print(constraints.to_markdown(index=False, numalign="left", stralign="left"))
display(constraints)



In [ ]:
'''asset_classes = {'Assets': ['JCI','TGT','CMCSA','CPB','MO','APA','MMC','JPM',
                            'ZION','PSA','BAX','BMY','LUV','PCAR','TXT','TMO',
                            'DE','MSFT','HPQ','SEE','VZ','CNP','NI','T','BA'], 
                 'Industry': ['Consumer Discretionary','Consumer Discretionary',
                              'Consumer Discretionary', 'Consumer Staples',
                              'Consumer Staples','Energy','Financials',
                              'Financials','Financials','Financials',
                              'Health Care','Health Care','Industrials','Industrials',
                              'Industrials','Health Care','Industrials',
                              'Information Technology','Information Technology',
                              'Materials','Telecommunications Services','Utilities',
                              'Utilities','Telecommunications Services','Financials']}

asset_classes = pd.DataFrame(asset_classes)
asset_classes = asset_classes.sort_values(by=['Assets'])

constraints = {'Disabled': [False, False, False, False, False],
               'Type': ['All Assets', 'Classes', 'Classes', 'Classes',
                        'Classes'],
               'Set': ['', 'Industry', 'Industry', 'Industry', 'Industry'],
               'Position': ['', 'Financials', 'Utilities', 'Industrials',
                            'Consumer Discretionary'],
               'Sign': ['<=', '<=', '<=', '<=', '<='],
               'Weight': [0.10, 0.2, 0.2, 0.2, 0.2],
               'Type Relative': ['', '', '', '', ''],
               'Relative Set': ['', '', '', '', ''],
               'Relative': ['', '', '', '', ''],
               'Factor': ['', '', '', '', '']}

constraints = pd.DataFrame(constraints)

display(constraints)
'''

In [ ]:
A, B = rp.assets_constraints(constraints, asset_classes)

### 4.2 Optimize the portfolio with the constraints

In [ ]:
port.ainequality = A
port.binequality = B

model = 'Classic'
rm = 'MV'
obj = 'Sharpe'
rf = 0

w = port.optimization(model=model, rm=rm, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

In [ ]:
ax = rp.plot_pie(w=w, title='Sharpe Mean Variance', others=0.05, nrow=25, cmap = "tab20",
                 height=6, width=10, ax=None)

In [ ]:
w_classes = pd.concat([asset_classes.set_index('Assets'), w], axis=1)

display(w_classes)

In [ ]:
w_classes = w_classes.groupby(['Industry']).sum()
w_classes = w_classes.drop(columns=['AI Beneficiary'])

display(w_classes)

### 3.5 Calculate Optimal Portfolios for Several Risk Measures with Constraints applied

In [ ]:
# Risk Measures available:
#
# 'MV': Standard Deviation.
# 'MAD': Mean Absolute Deviation.
# 'MSV': Semi Standard Deviation.
# 'FLPM': First Lower Partial Moment (Omega Ratio).
# 'SLPM': Second Lower Partial Moment (Sortino Ratio).
# 'CVaR': Conditional Value at Risk.
# 'EVaR': Entropic Value at Risk.
# 'WR': Worst Realization (Minimax)
# 'MDD': Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio).
# 'ADD': Average Drawdown of uncompounded cumulative returns.
# 'CDaR': Conditional Drawdown at Risk of uncompounded cumulative returns.
# 'EDaR': Entropic Drawdown at Risk of uncompounded cumulative returns.
# 'UCI': Ulcer Index of uncompounded cumulative returns.

rms = ['MV', 'MAD', 'MSV', 'FLPM', 'SLPM', 'CVaR',
       'EVaR', 'WR', 'MDD', 'ADD', 'CDaR', 'UCI', 'EDaR']

w_s = pd.DataFrame([])

for i in rms:
    w = port.optimization(model=model, rm=i, obj=obj, rf=rf, l=l, hist=hist)
    w_s = pd.concat([w_s, w], axis=1)
    
w_s.columns = rms

In [ ]:
w_s.style.format("{:.2%}").background_gradient(cmap='YlGn')

In [ ]:
import matplotlib.pyplot as plt

# Plotting a comparison of assets weights for each portfolio

fig = plt.gcf()
fig.set_figwidth(14)
fig.set_figheight(6)
ax = fig.subplots(nrows=1, ncols=1)

w_s.plot.bar(ax=ax)

### 3.6 Backtesting the strategy using VectorBT Pro

In [1]:
from vectorbtpro import *
import vectorbtpro as vbt
# whats_imported()

vbt.settings.set_theme("dark")

In [ ]:
vbt.phelp(vbt.Data.from_data)

In [2]:
# Tickers of assets
mag7 = ['AAPL', 'AMZN', 'GOOGL', 'META', 'MSFT', 'NVDA', 'TSLA']
AI_beneficiary = ['SNOW', 'DDOG', 'CRM']
sectors = ['XLC', 'XLB', 'XLV', 'XLRE', 'XLK', 'XLY', 'XLP', 'XLF', 'XLE', 'XLI', 'XLU', 'MID', 'EEM', 'GLD'] # S&P 500 sectors]
assets = sectors + AI_beneficiary #+ mag7
assets.sort()

In [ ]:
# Pull hourly data from Yahoo Finance

data = vbt.YFData.pull(
    assets,
    start='2023-03-24 UTC',
    end='2025-03-21 UTC',
    timeframe='1h',
    missing_columns='nan'
)

In [ ]:
# Pull daily data from Yahoo Finance

data = vbt.YFData.pull(
    assets,
    start='2016-01-03 UTC',
    end='2025-03-21 UTC',
    timeframe='D',
    missing_columns='nan'
)

In [ ]:
data.to_hdf('my_equity_data_hourly.h5')

In [3]:
data.to_hdf('my_equity_data_daily.h5')

NameError: name 'data' is not defined

In [ ]:
data = vbt.HDFData.pull('my_equity_data_hourly.h5')

In [4]:
data = vbt.HDFData.pull('my_equity_data_daily.h5')

In [5]:
close = data.get('Close')
display(close)

symbol,CRM,DDOG,EEM,GLD,MID,SNOW,XLB,XLC,XLE,XLF,XLI,XLK,XLP,XLRE,XLU,XLV,XLY
Date,,,,,,,,,,,,,,,,,
2016-01-04 00:00:00-05:00,76.283920,NaN,25.686905,102.889999,NaN,NaN,35.689854,NaN,41.514153,15.978041,44.399937,38.003075,39.210339,22.340847,32.299301,60.935844,69.578247
2016-01-05 00:00:00-05:00,76.622032,NaN,25.744328,103.180000,NaN,NaN,35.673153,NaN,41.672501,16.039581,44.518799,37.904190,39.462040,23.017817,32.531132,61.228756,69.487686
2016-01-06 00:00:00-05:00,75.866264,NaN,25.252087,104.669998,NaN,NaN,34.737892,NaN,40.068394,15.793446,43.831150,37.436680,39.328327,22.795586,32.471287,60.729057,68.808380
2016-01-07 00:00:00-05:00,73.887314,NaN,24.472702,106.150002,NaN,NaN,33.794300,NaN,39.090775,15.349043,42.642612,36.330837,38.856388,22.377520,32.254425,59.497078,67.395439
2016-01-08 00:00:00-05:00,72.823257,NaN,24.210173,105.680000,NaN,NaN,33.451946,NaN,38.588203,15.109752,42.209648,36.043144,38.557503,22.084135,32.239452,58.601078,66.670853
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-14 00:00:00-04:00,279.399994,101.800003,44.580002,275.239990,57.285999,156.110001,85.980003,96.580002,89.760002,48.520000,131.130005,213.940002,79.510002,41.450001,78.870003,144.929993,196.699997
2025-03-17 00:00:00-04:00,280.750000,103.970001,45.340000,276.730011,58.160999,156.389999,87.059998,97.269997,91.190002,49.090000,132.940002,215.429993,80.669998,42.180000,79.199997,146.770004,197.009995
2025-03-18 00:00:00-04:00,278.730011,102.930000,45.110001,279.959991,57.546001,154.360001,86.910004,95.910004,91.339996,49.029999,131.880005,212.149994,79.750000,41.919998,78.650002,146.839996,193.500000


In [6]:
returns = data.get("Close").vbt.to_returns()

#### Establish the constraints for the portfolio optimization.
In this example below we are establishing constraints in the following manner:
* Overweight the Materials, Healthcare, and Real Estate sector
* Benchmark Technology to have exposure to the Mag 7 stocks
* Gain exposure to Gold and Emerging Markets for hedging risk and diversification
* Gain exposure to stocks consider to be beneficiaries of AI

In [7]:


# Define assets and their corresponding industries

AI_Beneficiary =  ['No','No','No','No','No','No','No','No','No','No','No','No','No','No','Yes','Yes','Yes']

Industry =  [#'Technology', 'Consumer Discretionary', 'Communication Services', 'Communication Services', 'Technology', 'Technology', 'Consumer Discretionary', # Magnificent 7
                 'Communication Services', 'Materials', 'Healthcare', 'Real Estate',
                 'Technology', 'Consumer Discretionary', 'Consumer Staples', 'Financials', 'Energy', 'Industrials', 
                 'Utilities', 'Mid-Cap','Emerging Markets', 'Gold', 'Technology', 'Technology', 'Technology']

constraints = [
    {
        # Sector Overweighting (using "Classes" for the "Materials" sector)
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Materials",
        "Weight": 0.019,
        "Sign": ">="
    },
        # Sector Overweighting (using "Classes" for the "Healthcare" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Healthcare",
        "Weight": 0.101,
        "Sign": ">="
    },
        # Sector Overweighting (using "Classes" for the "Real Estate" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Real Estate",
        "Weight": 0.023,
        "Sign": ">="
    },
        # Sector Benchmarking (using "Classes" for the "Technology" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Technology",
        "Weight": 0.29,
        "Sign": ">="
    },
        # Sector Overweighting (using "Classes" for the "Financials" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Financials",
        "Weight": 0.136,
        "Sign": "<="
    },
        # Sector equal weighting (using "Classes" for the "Energy" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Energy",
        "Weight": 0.05,
        "Sign": "<="
    },
        # Sector equal weighting (using "Classes" for the "Communication Services" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Communication Services",
        "Weight": 0.082,
        "Sign": "<="
    },
        # Sector equal weighting (using "Classes" for the "Consumer Discretionary" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Consumer Discretionary",
        "Weight": 0.113,
        "Sign": "<="
    },
        # Sector exposure to MidCap stocks (using "Classes" for the "Mid-Cap" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Mid-Cap",
        "Weight": 0.1,
        "Sign": ">="
    },
        # Sector exposure to Emerging Markets (using "Classes" for the "Emerging Markets" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Emerging Markets",
        "Weight": 0.05,
        "Sign": ">="
    },
        # Sector exposure to Gold (using "Classes" for the "Gold" sector)
    {
        "Type": "Classes",
        "Set": "Industry",
        "Position": "Gold",
        "Weight": 0.05,
        "Sign": ">="
    },
        # AI Beneficiary exposure (using "Classes" for the "AI Beneficiary" class)
    {
        "Type": "Classes",
        "Set": "AI Beneficiary",
        "Position": "Yes",
        "Weight": vbt.Param([0.015], name="AI_Beneficiary_maxw") , # or 0.015 if not using vbt.Param
        "Sign": ">="
    },
        # Exposure to all stocks in the AI Beneficiary class (using "Each asset in a class" for the "AI Beneficiary" class)
    {
        "Type": "Each asset in a class",
        "Set": "AI Beneficiary",
        "Position": "Yes",
        "Weight": 0.02,
        "Sign": "<="
    }
]

#### Estimate the optimized portfolio and backtest to produce the overall returns for a single Risk Measure - Mean Variance

In [ ]:
# Estimate optimal portfolio:

model='Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
rm = 'CVaR' # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk free rate
l = 0 # Risk aversion factor, only useful when obj is 'Utility'

pfo = vbt.PortfolioOptimizer.from_riskfolio(
    returns=returns,
    #every=vbt.Param(["W","2W","M","Q"], name="rebalance_every"),
    every = "MS",
    start="2023-04-01",
    #lookback_period=vbt.Param(["3M"], name="lookback_period"),
    model=model,
    method_mu="hist", 
    method_cov="hist",
    asset_classes={"Industry": Industry, "AI Beneficiary": AI_Beneficiary},
    constraints=constraints,
    d=0.94,
    rm=rm,
    obj=obj,
    rf=0, 
    l=0, 
    hist=True,
    param_search_kwargs=dict(incl_types=list) # Include all types of parameters
)

#pfo.plot(per_column=True).show_svg()

In [ ]:
pfo.plot().show()

In [ ]:
pfo.allocations.groupby("AI_Beneficiary_maxw").max()

In [ ]:
pfo.stats()

In [ ]:
print(pfo.allocations)

In [ ]:
print(pfo.alloc_records.records_readable)

In [ ]:
pf = vbt.Portfolio.from_optimizer(close=data, optimizer=pfo, size_type='targetpercent',
                                  init_cash = 100000,
                                  freq="1h")

pf.sharpe_ratio

In [ ]:
vbt.phelp(vbt.Portfolio.from_optimizer)

In [ ]:
pf.get_asset_value(group_by=False).vbt / pf.value

In [ ]:
pf.stats(per_column=True)

In [ ]:
pf.get_beta()

In [ ]:
pf.get_alpha()

In [ ]:
pf.get_omega_ratio()

In [ ]:
trade_history = pf.get_trade_history()
display(trade_history)

In [ ]:

print(trade_history.columns)

In [ ]:
pf.plot_trades(column='XLC').show_svg()

In [ ]:
stats_df = pf.stats(per_column=True)
print(stats_df)

In [ ]:
pf.plot(subplots="all", per_column=True).show_svg()

In [ ]:
# Risk Measures available:
#
# 'MV': Standard Deviation.
# 'MAD': Mean Absolute Deviation.
# 'MSV': Semi Standard Deviation.
# 'FLPM': First Lower Partial Moment (Omega Ratio).
# 'SLPM': Second Lower Partial Moment (Sortino Ratio).
# 'CVaR': Conditional Value at Risk.
# 'EVaR': Entropic Value at Risk.
# 'WR': Worst Realization (Minimax)
# 'MDD': Maximum Drawdown of uncompounded cumulative returns (Calmar Ratio).
# 'ADD': Average Drawdown of uncompounded cumulative returns.
# 'CDaR': Conditional Drawdown at Risk of uncompounded cumulative returns.
# 'EDaR': Entropic Drawdown at Risk of uncompounded cumulative returns.
# 'UCI': Ulcer Index of uncompounded cumulative returns.

model = 'Classic' # Could be Classic (historical), BL (Black Litterman) or FM (Factor Model)
hist = True # Use historical scenarios for risk measures that depend on scenarios
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility or Sharpe

rms = ['MV', 'MAD', 'MSV', 'FLPM', 'SLPM', 'CVaR',
      'EVaR', 'WR', 'MDD', 'ADD', 'CDaR', 'UCI', 'EDaR']


pfo = vbt.PortfolioOptimizer.from_riskfolio(
        returns=returns,
        every=vbt.Param(["W", "2W", "M"], name="rebalance_every"),
        model=model,
        method_mu="hist", 
        method_cov="hist",
        asset_classes={"Industry": Industry, "AI Beneficiary": AI_Beneficiary},
        constraints=constraints,                      # Constraints used in this iteration
        d=0.94,
        rm=vbt.Param(rms, name="Risk Measure"),       # Risk measures used in this iteration
        obj=obj,
        rf=0, 
        l=0, 
        hist=hist,
        param_search_kwargs=dict(incl_types=list) # Include all types of parameters
    )
pf = vbt.Portfolio.from_optimizer(close=data, optimizer=pfo, size_type='targetpercent',
                                      init_cash = 100000,
                                      freq="1h")


  0%|          | 0/39 [00:02<?, ?it/s, rebalance_every=W, AI_Beneficiary_maxw=0.015, Risk Measure=MV]

 10%|#         | 49/480 [00:02<00:17, 24.29it/s, 2016-12-12 → 2016-12-16]

In [ ]:
vbt.phelp(pf.stats)

In [ ]:
# Assuming 'pf' is a portfolio object
# Calculate the total number of trades for each position
total_trades_stats = pf.stats(metrics="total_trades", agg_func=None, settings=dict(trades_type="positions"), per_column=True)

# Display the calculated statistics
print(total_trades_stats)

In [9]:
stats_df = pf.stats(per_column=True)
print(stats_df)

rebalance_every                                      W  \
AI_Beneficiary_maxw                              0.015   
Risk Measure                                        MV   
Start Index                  2016-01-04 00:00:00-05:00   
End Index                    2025-03-20 00:00:00-04:00   
Total Duration                        96 days 13:00:00   
Start Value                                   100000.0   
Min Value                                 92506.949266   
Max Value                                188848.835073   
End Value                                 179702.86606   
Total Return [%]                             79.702866   
Benchmark Return [%]                        152.798583   
Position Coverage [%]                        50.237376   
Max Gross Exposure [%]                           100.0   
Max Drawdown [%]                             30.703487   
Max Drawdown Duration                 25 days 01:00:00   
Total Orders                                      2970   
Total Fees Pai

In [21]:
stats_df.index

Index(['Start Index', 'End Index', 'Total Duration', 'Start Value',
       'Min Value', 'Max Value', 'End Value', 'Total Return [%]',
       'Benchmark Return [%]', 'Position Coverage [%]',
       'Max Gross Exposure [%]', 'Max Drawdown [%]', 'Max Drawdown Duration',
       'Total Orders', 'Total Fees Paid', 'Total Trades', 'Win Rate [%]',
       'Best Trade [%]', 'Worst Trade [%]', 'Avg Winning Trade [%]',
       'Avg Losing Trade [%]', 'Avg Winning Trade Duration',
       'Avg Losing Trade Duration', 'Profit Factor', 'Expectancy',
       'Sharpe Ratio', 'Calmar Ratio', 'Omega Ratio', 'Sortino Ratio'],
      dtype='object')

In [22]:
#stats_df.shape
stats_df.columns

MultiIndex([( 'W', 0.015,   'MV'),
            ( 'W', 0.015,  'MAD'),
            ( 'W', 0.015,  'MSV'),
            ( 'W', 0.015, 'FLPM'),
            ( 'W', 0.015, 'SLPM'),
            ( 'W', 0.015, 'CVaR'),
            ( 'W', 0.015, 'EVaR'),
            ( 'W', 0.015,   'WR'),
            ( 'W', 0.015,  'MDD'),
            ( 'W', 0.015,  'ADD'),
            ( 'W', 0.015, 'CDaR'),
            ( 'W', 0.015,  'UCI'),
            ( 'W', 0.015, 'EDaR'),
            ('2W', 0.015,   'MV'),
            ('2W', 0.015,  'MAD'),
            ('2W', 0.015,  'MSV'),
            ('2W', 0.015, 'FLPM'),
            ('2W', 0.015, 'SLPM'),
            ('2W', 0.015, 'CVaR'),
            ('2W', 0.015, 'EVaR'),
            ('2W', 0.015,   'WR'),
            ('2W', 0.015,  'MDD'),
            ('2W', 0.015,  'ADD'),
            ('2W', 0.015, 'CDaR'),
            ('2W', 0.015,  'UCI'),
            ('2W', 0.015, 'EDaR'),
            ( 'M', 0.015,   'MV'),
            ( 'M', 0.015,  'MAD'),
            ( 'M', 0

#### Find the Highest total return for a given Max setting for AI_Beneficiary across all available Risk Measures

In [ ]:
# Get unique values of AI_Beneficiary_maxw
ai_beneficiary_values = stats_df.columns.get_level_values('AI_Beneficiary_maxw').unique()

# Find the 'Risk Measure' with the highest "Total Return [%]" for each AI_Beneficiary_maxw
results = {}
for value in ai_beneficiary_values:
    # Select the columns for the current AI_Beneficiary_maxw
    subset = stats_df.xs(value, level='AI_Beneficiary_maxw', axis=1).loc['Total Return [%]']
    
    # Find the 'Risk Measure' with the highest "Total Return [%]"
    max_risk_measure = subset.idxmax()
    max_return = subset.max()
    
    # Store the result
    results[value] = (max_risk_measure, max_return)

# Display the results
for value, (risk_measure, max_return) in results.items():
    print(f"AI_Beneficiary_maxw = {value}: Highest Total Return [%] = {max_return} for Risk Measure = {risk_measure}")

AI_Beneficiary_maxw = 0.015: Highest Total Return [%] = 86.17017468638961 for Risk Measure = ('W', 'MSV')


#### Find the Highest Sharpe Ratio for a given Max setting for AI_Beneficiary across all available Risk Measures

In [25]:
# Get unique values of AI_Beneficiary_maxw
ai_beneficiary_values = stats_df.columns.get_level_values('AI_Beneficiary_maxw').unique()

# Find the 'Risk Measure' with the highest "Total Return [%]" for each AI_Beneficiary_maxw
results = {}
for value in ai_beneficiary_values:
    # Select the columns for the current AI_Beneficiary_maxw
    subset = stats_df.xs(value, level='AI_Beneficiary_maxw', axis=1).loc['Sharpe Ratio']
    
    # Find the 'Risk Measure' with the highest "Total Return [%]"
    max_risk_measure = subset.idxmax()
    max_ratio = subset.max()
    
    # Store the result
    results[value] = (max_risk_measure, max_ratio)

# Display the results
for value, (risk_measure, max_ratio) in results.items():
    print(f"AI_Beneficiary_maxw = {value}: Highest Sharpe ratio = {max_ratio} for Risk Measure = {risk_measure}")

AI_Beneficiary_maxw = 0.015: Highest Sharpe ratio = 3.446737667544696 for Risk Measure = ('W', 'MSV')


#### Print the stats for a particular index in the MultiIndex dataframe

In [15]:
# Access a specific value for AI_Beneficiary_maxw = 0.05 and Risk Measure = 'CVaR'
specific_value = stats_df.loc[:, ('W', 0.015, 'EDaR')]

# Display the specific value
print("Specific value for AI_Beneficiary_maxw = 0.015 and Risk Measure = 'EDaR':")
print(specific_value)

Specific value for AI_Beneficiary_maxw = 0.015 and Risk Measure = 'EDaR':
Start Index                    2016-01-04 00:00:00-05:00
End Index                      2025-03-20 00:00:00-04:00
Total Duration                          96 days 13:00:00
Start Value                                     100000.0
Min Value                                   97950.591949
Max Value                                  176849.495385
End Value                                  169824.389897
Total Return [%]                                69.82439
Benchmark Return [%]                          152.798583
Position Coverage [%]                          50.237376
Max Gross Exposure [%]                             100.0
Max Drawdown [%]                               28.219301
Max Drawdown Duration                   26 days 03:00:00
Total Orders                                        2970
Total Fees Paid                                      0.0
Total Trades                                        1494
Win Rate [%]  

#### Print the key ratios for each value of the rebalance frequency ('W", "2W', ..) and AI Beneficiary max percentage allocations

In [32]:
# Get unique values of rebalance_every
rebalance_values = stats_df.columns.get_level_values('rebalance_every').unique()
ai_beneficiary_values = stats_df.columns.get_level_values('AI_Beneficiary_maxw').unique()

# Find the 'Risk Measure' with the highest "Total Return [%]" for each rebalance_every
results = {}
for value in rebalance_values:
    for ai_value in ai_beneficiary_values:
        try:
            # Select the columns for the current rebalance_every and AI_Beneficiary_maxw
            subset = stats_df.xs((value, ai_value), level=('rebalance_every', 'AI_Beneficiary_maxw'), axis=1).loc['Total Return [%]']

            # Find the 'Risk Measure' with the highest "Total Return [%]"
            max_risk_measure = subset.idxmax()
            max_return = subset.max()

            # Get the other metrics for the selected 'Risk Measure'
            sharpe_ratio = stats_df.loc['Sharpe Ratio', (value, ai_value, max_risk_measure)]
            calmar_ratio = stats_df.loc['Calmar Ratio', (value, ai_value, max_risk_measure)]
            omega_ratio = stats_df.loc['Omega Ratio', (value, ai_value, max_risk_measure)]

            # Store the result
            results[(value, ai_value)] = (max_risk_measure, max_return, sharpe_ratio, calmar_ratio, omega_ratio)
        except KeyError:
            continue

# Display the results
for (value, ai_value), (risk_measure, max_return, sharpe_ratio, calmar_ratio, omega_ratio) in results.items():
    print(f"rebalance_every = {value}, AI_Beneficiary_maxw = {ai_value}:")
    print(f"  Highest Total Return [%] = {max_return} for Risk Measure = {risk_measure}")
    print(f"  Sharpe Ratio = {sharpe_ratio}")
    print(f"  Calmar Ratio = {calmar_ratio}")
    print(f"  Omega Ratio = {omega_ratio}")
    print()

rebalance_every = W, AI_Beneficiary_maxw = 0.015:
  Highest Total Return [%] = 86.17017468638961 for Risk Measure = MSV
  Sharpe Ratio = 3.446737667544696
  Calmar Ratio = 36.27385581987066
  Omega Ratio = 1.1517712732838525

rebalance_every = 2W, AI_Beneficiary_maxw = 0.015:
  Highest Total Return [%] = 72.22585664102496 for Risk Measure = CVaR
  Sharpe Ratio = 3.1646412239704254
  Calmar Ratio = 26.336124847421463
  Omega Ratio = 1.1376945038455901

rebalance_every = M, AI_Beneficiary_maxw = 0.015:
  Highest Total Return [%] = 37.54140474321948 for Risk Measure = EVaR
  Sharpe Ratio = 1.9349805648697147
  Calmar Ratio = 7.240419023836564
  Omega Ratio = 1.0819864501068086



In [ ]:
stats_df.index

In [ ]:
pf.plot(subplots="all", per_column=True).show_svg()